# プログラミング言語でアキネイター

In [158]:
import requests
from bs4 import BeautifulSoup
import json
import pathlib
import dotenv
import copy
from matplotlib import pyplot
import seaborn as sns
import japanize_matplotlib
import sklearn.tree
from joblib import delayed, Parallel
import pandas as pd
import io
from tqdm.auto import tqdm

In [155]:
dotenv.load_dotenv() # OPENAI_API_KEY

True

## データダウンロード

In [3]:
rootpath = pathlib.Path('../data') / 'programming_languages'
datapath = rootpath / 'source'
datapath.mkdir(parents=True, exist_ok=True)

In [4]:
response = requests.get('https://ja.wikipedia.org/wiki/プログラミング言語一覧')

In [5]:
soup = BeautifulSoup(response.content)

In [11]:
headlines = soup.find_all(class_="mw-headline")

In [16]:
anchor.find

[]

In [60]:
anchor.nextSibling

'\n'

In [65]:
headlines[-10].parent

<h2><span id=".E3.81.8B.E8.A1.8C"></span><span class="mw-headline" id="か行">か行</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E%E4%B8%80%E8%A6%A7&amp;action=edit&amp;section=28" title="節を編集: か行">編集</a><span class="mw-editsection-bracket">]</span></span></h2>

In [64]:
headlines[-10].parent.findNextSibling("h2")

<h2><span id=".E3.81.95.E8.A1.8C"></span><span class="mw-headline" id="さ行">さ行</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E%E4%B8%80%E8%A6%A7&amp;action=edit&amp;section=29" title="節を編集: さ行">編集</a><span class="mw-editsection-bracket">]</span></span></h2>

In [63]:
headlines[-10].parent

<h2><span id=".E3.81.8B.E8.A1.8C"></span><span class="mw-headline" id="か行">か行</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E%E4%B8%80%E8%A6%A7&amp;action=edit&amp;section=28" title="節を編集: か行">編集</a><span class="mw-editsection-bracket">]</span></span></h2>

In [50]:
headlines[-2].parent.next

<span id=".E3.82.8F.E8.A1.8C"></span>

In [73]:
headlines[0].get("tag")

In [74]:
anchor.name

'ul'

In [84]:
headline = headlines[-2]
for sibling in headline.parent.next_siblings:
    print(sibling)
    if sibling.name == "ul":
        anchor = sibling
        break            
    if str(sibling).find("mw-headline") >= 0: # 次のheadline
        anchor = None
        break
print(anchor is None)




<h2><span id=".E9.96.A2.E9.80.A3.E9.A0.85.E7.9B.AE"></span><span class="mw-headline" id="関連項目">関連項目</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E%E4%B8%80%E8%A6%A7&amp;action=edit&amp;section=37" title="節を編集: 関連項目">編集</a><span class="mw-editsection-bracket">]</span></span></h2>
True


In [109]:
urls = {}
for headline in headlines:
    if headline.text.find("関連項目") >= 0:
        continue

    # ulを探す
    anchor = None
    for sibling in headline.parent.next_siblings:
        if sibling.name == "ul":
            anchor = sibling
            break            
        if str(sibling).find("mw-headline") >= 0: # 次のheadline
            anchor = None
            break
    if anchor is None:
        continue
    for element in anchor.findChildren("li"):
        a = element.findChild("a")
        if (a is None) or (a["href"].find("edit") >= 0): #記事がない
            continue
        if element.text.find("mw-headline") >= 0: # 次のheadline
            continue
            
        # print("\n")
        # print(element)
        # print(a["href"])
        urls[element.text] = a["href"]
        # if str(element).find("mw-headline") >= 0:
        #     break
        # for t in element.text.split("\n"):
        #     print(t)
    print(headline)
    print(len(urls))


<span class="mw-headline" id="A">A</span>
16
<span class="mw-headline" id="B">B</span>
29
<span class="mw-headline" id="C">C</span>
60
<span class="mw-headline" id="D">D</span>
65
<span class="mw-headline" id="E">E</span>
74
<span class="mw-headline" id="F">F</span>
86
<span class="mw-headline" id="G">G</span>
91
<span class="mw-headline" id="H">H</span>
103
<span class="mw-headline" id="I">I</span>
113
<span class="mw-headline" id="J">J</span>
123
<span class="mw-headline" id="K">K</span>
128
<span class="mw-headline" id="L">L</span>
141
<span class="mw-headline" id="M">M</span>
163
<span class="mw-headline" id="N">N</span>
169
<span class="mw-headline" id="O">O</span>
180
<span class="mw-headline" id="P">P</span>
204
<span class="mw-headline" id="Q">Q</span>
204
<span class="mw-headline" id="R">R</span>
216
<span class="mw-headline" id="S">S</span>
241
<span class="mw-headline" id="T">T</span>
251
<span class="mw-headline" id="U">U</span>
254
<span class="mw-headline" id="V">V</span>

In [36]:
import urllib.parse

In [125]:
response = requests.get("http://ja.wikipedia.org/wiki/A+")

In [127]:
response.history

[<Response [301]>]

In [120]:
response = requests.get('http://ja.wikipedia.org/wiki/A_(%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E)', allow_redirects=False)

In [128]:
response = requests.get('https://ja.wikipedia.org/wiki/A_(%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E)')

In [130]:
BeautifulSoup(response.content).find("title")

<title>Brainfuck - Wikipedia</title>

In [141]:
data = {}

In [142]:
for key, url in tqdm(list(urls.items())):
    filepath = f'{key.replace("/", "_")}.json'
    if filepath in data:
        continue
    response = requests.get(f'https://ja.wikipedia.org/{url}')
    soup = BeautifulSoup(response.content)
    # print(soup.find("title"), key, soup.find("title").text.find(key))
    if soup.find("title").text.find(key) == -1: #独立したページがない
        continue
    texts_newlines = soup.find(class_="mw-body-content").text.split("\n")
    texts = []
    for text_newline in texts_newlines:
        for text in text_newline.split("。"):
            if len(text):
                texts.append(text)
    json.dump(dict(
        url=url,
        name=key,
        texts=texts,
    ), (datapath / filepath).open('w+'))

    data[filepath] = dict(
        url=url,
        name=key,
        texts=texts,
    )
    

100%|██████████| 276/276 [02:53<00:00,  1.59it/s]


In [143]:
import sys

In [145]:
sys.path.append("..")

In [147]:
import importlib

In [160]:
importlib.reload(llmakinator)

<module 'llmakinator' from '/Users/keisuke.ogaki/ghq/github.com/Hi-king/chatgpt_akinator/notebooks/../llmakinator/__init__.py'>

In [146]:
import llmakinator

In [149]:
data_restricted = {
    key: dict(
        name=datum["name"],
        text=llmakinator.preprocess.filter_first_k_texts(datum["texts"], 500)  
    ) for key, datum in data.items()
}

In [151]:
import random
random_samples = random.sample(list(data_restricted.values()), 7)

In [166]:
questions = llmakinator.questions.generate_questions(
    world = 'プログラミング言語',
    data = random_samples,
    question_num = 20,    
)

[
    "オブジェクト指向言語ですか？",
    "関数型言語ですか？",
    "手続き型言語ですか？",
    "静的型付け言語ですか？",
    "動的型付け言語ですか？",
    "コンパイル言語ですか？",
    "インタプリタ言語ですか？",
    "マルチパラダイム言語ですか？",
    "ガベージコレクション機能がありますか？",
    "クロスプラットフォームですか？",
    "シェーディング言語ですか？",
    "統計解析に特化した言語ですか？",
    "グラフィカルユーザインターフェイスの開発に適していますか？",
    "数学的な処理に適していますか？",
    "Web開発に適していますか？",
    "並列処理に適していますか？",
    "マクロ機能がありますか？",
    "オープンソースですか？",
    "商用ライセンスが必要ですか？",
    "日本語のドキュメントが充実していますか？"
]


In [167]:
questions

In [159]:
dumppath = rootpath / 'qanda'
dumppath.mkdir(exist_ok=True)

In [162]:
data_with_qanda = copy.deepcopy(data_restricted)

In [168]:
@delayed
def qandq_each(key: str, datum):
    if 'qanda' in data_with_qanda[key]:
        return
    answer = llmakinator.answers.answer_datum(
        datum=datum, 
        questions=questions, 
        world='プログラミング言語'
    )
    
    df_qanda = pd.DataFrame(answer).assign(**{"質問": questions})
    data_with_qanda[key]['qanda'] = df_qanda.to_dict(orient="list")
    json.dump(data_with_qanda[key], (dumppath / key).open("w+"))

results = Parallel(n_jobs=5)(
    qandq_each(key, datum) for key, datum in tqdm(list(data_restricted.items()))
)





 11%|█▏        | 24/213 [01:25<11:13,  3.56s/it]


TypeError: 'NoneType' object is not iterable